# Binary image classifier with:
* TPU / Multi-GPU ready code
* Dataset created from directories with separate classes
* Preprocessing and augmentation as a Keras layer in dataset preprocessor
* Simple baseline CNN
* Transfer learning based on Efficient-NET
* Builds on:

https://www.kaggle.com/code/donkeys/keras-binary-cats-dogs-resnet-98

https://towardsdatascience.com/a-comprehensive-guide-to-training-cnns-on-tpu-1beac4b0eb1c

In [ ]:
# Initial imports
import tensorflow as tf
import keras_preprocessing
from keras_preprocessing import image
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import layers
import tensorflow_hub as hub
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import random
import math
import PIL

In [ ]:
try: 
    # For use with TPU:

    # Detect TPUs
    
    # Locate TPUs on the network
    # tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect() # TPU detection
    
    # TPUStrategy contains the necessary distributed training code that will work on TPUs 
    # with their 8 compute cores
    # strategy = tf.distribute.TPUStrategy(tpu)
    
    # Multi GPU training
    strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0"]) #, "/gpu:1"])

except ValueError: # If TPU or GPU is not available
    strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU

In [ ]:
print(f'Number of accelerators: {strategy.num_replicas_in_sync}')

In [ ]:
PATH_IMAGES = './data/PetImages'

In [ ]:
!ls $PATH_IMAGES

In [ ]:
BATCH_SIZE = 32 * strategy.num_replicas_in_sync

AUTOTUNE = tf.data.AUTOTUNE

# This is related to the feature size optimization, a multiple of 128 required for TPU
IMG_SIZE = 128 * 2

In [ ]:
train_val_dir = PATH_IMAGES
train_val_cat_files = os.listdir(PATH_IMAGES + '/Cat')
train_val_dog_files = os.listdir(PATH_IMAGES + '/Dog')

# Add a set for final model testing if needed
# test_dir = 

In [ ]:
TRAIN_TOTAL = len(train_val_cat_files) + len(train_val_dog_files)

In [ ]:
CAT = 'cat'
DOG = 'dog'

In [ ]:
labels = []
df_data = pd.DataFrame()

In [ ]:
TRAIN_TOTAL

In [ ]:
%%time
idx = 0
img_sizes = []
file_dir = []
files_str = []
widths = np.zeros(TRAIN_TOTAL, dtype=int)
heights = np.zeros(TRAIN_TOTAL, dtype=int)
aspect_ratio = np.zeros(TRAIN_TOTAL)

for filename in train_val_cat_files:
    labels.append(CAT)
    filename_str = f'{PATH_IMAGES}/Cat/{filename}'
    files_str.append(filename_str)
    # Standardise some old files
    img = PIL.Image.open(filename_str).convert('RGB')
    file_dir.append(f'{PATH_IMAGES}/Cat/')
    img_size = img.size
    img_sizes.append(img_size)
    widths[idx] = img_size[0]
    heights[idx] = img_size[1]
    aspect_ratio[idx] = img_size[0]/img_size[1]
    
    # We can resize already here if we want
    #img = img.resize((IMG_SIZE, IMG_SIZE))
    #img.save(filename_str)
    
    idx+=1
    
for filename in train_val_dog_files:
    labels.append(DOG)
    filename_str = f'{PATH_IMAGES}/Dog/{filename}'
    files_str.append(filename_str)
    # Standardise some old files
    img = PIL.Image.open(filename_str).convert('RGB')
    file_dir.append(f'{PATH_IMAGES}/Dog/')
    img_size = img.size
    img_sizes.append(img_size)
    widths[idx] = img_size[0]
    heights[idx] = img_size[1]
    aspect_ratio[idx] = img_size[0]/img_size[1]
    
    # We can resize already here if we want
    #img = img.resize((IMG_SIZE, IMG_SIZE))
    #img.save(filename_str)
    
    idx+=1

In [ ]:
file_list = train_val_cat_files + train_val_dog_files

In [ ]:
len(labels)

# Creating dataset dataframe from directory

In [ ]:
df_data['filename'] = file_list
df_data['filedir'] = file_dir
df_data['cat_or_dog'] = labels
df_data['files_str'] = files_str
label_encoder = LabelEncoder()
df_data['cd_label'] = label_encoder.fit_transform(df_data['cat_or_dog'])
df_data["size"] = img_sizes
df_data["width"] = widths
df_data["height"] = heights
df_data["aspect_ratio"] = aspect_ratio
# df_data.head()

In [ ]:
# Sorting by aspect ratio to detect some edge case shapes
df_sorted = df_data.sort_values(by='aspect_ratio')

In [ ]:
def plot_first_9(df_to_plot):
    plt.figure(figsize=[30, 30])
    for x in range(9):
        filename = df_to_plot.iloc[x]['filename']
        path_to_plot = df_to_plot.iloc[x]['filedir'] + df_to_plot.iloc[x]['filename']
        img = PIL.Image.open(path_to_plot)
        print(filename)
        plt.subplot(3, 3, x + 1)
        plt.imshow(img)
        title_str = filename+" "+str(df_to_plot.iloc[x].aspect_ratio)
        plt.title(title_str)

In [ ]:
# plot_first_9(df_sorted)

In [ ]:
# Dropping wrong samples
df_sorted=df_sorted[:-3]

In [ ]:
def convert_to_image(filename):
    img = tf.io.read_file(filename)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.image.resize(img, (IMG_SIZE, IMG_SIZE))
    return img

In [ ]:
def convert_to_images_labels(filename, label):
    return convert_to_image(filename), label

In [ ]:
def prepare_for_training(ds, data_augement_fn=None):
    ds = ds.map(convert_to_images_labels,
               num_parallel_calls=AUTOTUNE)

    # Only for datasets fitting in memmory
    # ds = ds.cache() # Important to do before data aug
    
    # Big buffer size preferred
    ds = ds.shuffle(buffer_size=2048)
    
    # Infinite dataset
    ds = ds.repeat()
    
    ds = ds.batch(BATCH_SIZE)
    
    # Apply data augmentation
    if data_augement_fn:
        ds = ds.map(data_augement_fn,
                   num_parallel_calls=AUTOTUNE)
        
    ds = ds.prefetch(buffer_size=AUTOTUNE)
    
    return ds

In [ ]:
def plot_learning_curves(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(len(acc))
    
    plt.plot(epochs, acc, 'r', label='Training accuracy')
    plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
    plt.title('Trainig and validation accuracy')
    plt.legend(loc=0)
    plt.figure();
    
    plt.show();

In [ ]:
# Randomizing dataframe rows
df_sorted=df_sorted.sample(frac=1)
df_sorted=df_sorted.sample(frac=1)

In [ ]:
df_sorted.head()

# Train val split

In [ ]:
train_val_split = int(0.25 * len(df_sorted))

In [ ]:
df_train = df_sorted[:-train_val_split]

In [ ]:
df_val = df_sorted[-train_val_split:]

In [ ]:
train_files_ds = tf.data.Dataset.from_tensor_slices(df_train['files_str'])
val_files_ds = tf.data.Dataset.from_tensor_slices(df_val['files_str'])

train_labels_ds = tf.data.Dataset.from_tensor_slices(df_train['cd_label'])
val_labels_ds = tf.data.Dataset.from_tensor_slices(df_val['cd_label'])

In [ ]:
train_list_ds = tf.data.Dataset.zip((train_files_ds, train_labels_ds))

val_list_ds = tf.data.Dataset.zip((val_files_ds, val_labels_ds))

In [ ]:
# See a piece of the files dataset with labels
for sample in train_list_ds.take(5):
    print(sample[0].numpy(), sample[1].numpy())

# Converting files TF dataset into pictures dataset|

In [ ]:
train_ds = prepare_for_training(train_list_ds)
val_ds = prepare_for_training(val_list_ds)

In [ ]:
train_ds.options

# For an infinite dataset training (ds.repeat()) one has to set 
* steps_per_epoch
* validation_steps

# Note: remember to tune batch size for TPU and learning rate accordingly to the (large) batch size (not done here)

In [ ]:
steps_per_epoch = math.ceil(len(train_list_ds)/BATCH_SIZE)
validation_steps = math.ceil(len(val_list_ds)/BATCH_SIZE)

# A simple baseline CNN model

In [ ]:
def define_model(show_summary=False):
    
    model = tf.keras.models.Sequential([
        
        tf.keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3)),
        
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D(2,2),
        
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D(2,2),
        
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D(2,2),
        
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D(2,2),
        
        layers.GlobalAveragePooling2D(),
        layers.Dropout(0.5),
        
        layers.Dense(512, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])
    
    if show_summary:
        model.summary()
          
    model.compile(loss='binary_crossentropy', 
                  optimizer='adam', 
                  metrics=['accuracy'],
                 steps_per_execution=32)
    
    return model

# Creating model in the distributed strategy scope

In [ ]:
with strategy.scope():
    model = define_model(show_summary=True)

# Training

In [ ]:
history = model.fit(train_ds,
                   steps_per_epoch=steps_per_epoch,
                   epochs=5,
                   validation_data=val_ds,
                   validation_steps=validation_steps,
                   verbose=1)

In [ ]:
model.evaluate(val_ds, steps=validation_steps)

In [ ]:
plot_learning_curves(history)

# Training with data augmentation

# Data augmentation placed outside model in the data pipeline, because TPU may usually not support augmentation ops

In [ ]:
data_augmentation = tf.keras.Sequential([
    layers.experimental.preprocessing.RandomFlip('horizontal'),
    layers.experimental.preprocessing.RandomRotation(0.2),
    layers.experimental.preprocessing.RandomZoom(0.2),
    layers.experimental.preprocessing.RandomContrast(factor=0.2),
])

def data_augment(img, label):
    return data_augmentation(img), label

train_ds = prepare_for_training(train_list_ds, 
                               data_augement_fn=data_augment)

In [ ]:
# Early stopping callback automatically retrieving best weights
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=2,
                                                    restore_best_weights=True)

In [ ]:
history = model.fit(train_ds, 
                    steps_per_epoch=steps_per_epoch,
                    epochs=5, 
                    validation_data=val_ds,
                    validation_steps=validation_steps,
                    verbose=1,
                    callbacks=[early_stopping_cb])

# Transfer learning based on EfficientNet

In [ ]:
# TPU has no access to local drive so one has to use uncompressed model
# loaded directly to TPU

os.environ["TFHUB_MODLE_LOAD_FORMAT"] = "UNCOMPRESSED"

efficientnet_url = "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_b0/classification/2"

In [ ]:
def create_feature_vectors_model(model_url):
    feature_extractor_layer = hub.KerasLayer(model_url,
                                            trainable=False,
                                            name='feature_extraction_layer')
    
    model = tf.keras.Sequential([
        feature_extractor_layer,
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid', name='output_layer')
    ])
    
    model.build([None, IMG_SIZE, IMG_SIZE, 3])
    
    model.summary()
    
    model.compile(loss='binary_crossentropy',
                 optimizer='adam',
                 metrics=['accuracy'],
                 steps_per_execution=32)
    
    return model

In [ ]:
with strategy.scope():
    model = create_feature_vectors_model(efficientnet_url)

In [ ]:
history = model.fit(train_ds,
                   steps_per_epoch=steps_per_epoch,
                   epochs=3,
                   validation_data=val_ds,
                   validation_steps=validation_steps,
                   verbose=1,
                   callbacks=[early_stopping_cb])